In [4]:
import pandas as pd

# Load the first dataset
df1 = pd.read_csv('E:\Excelr-P269-Internship-Project-Group2-OIl-Price-Prediction-And-Forecasting-Using-Python\Dataset\BrentOilPrices.csv')

# Load the second dataset
df2 = pd.read_csv('E:\Excelr-P269-Internship-Project-Group2-OIl-Price-Prediction-And-Forecasting-Using-Python\Dataset\oil_prices_yahoo1.csv')

# renaming the column to price
df2.rename(columns = {'Close':'Price'}, inplace = True)

# Concatenate the two datasets
result = pd.concat([df1, df2]).drop_duplicates(subset=['Date'], keep='first')

# Save the result to a new CSV file
result.to_csv('E:\Excelr-P269-Internship-Project-Group2-OIl-Price-Prediction-And-Forecasting-Using-Python\Dataset\Oil_Prices.csv', index=False)


In [7]:
df = pd.read_csv('E:\Excelr-P269-Internship-Project-Group2-OIl-Price-Prediction-And-Forecasting-Using-Python\Dataset\Oil_Prices.csv')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14762 entries, 0 to 14761
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    14762 non-null  object 
 1   Price   14762 non-null  float64
dtypes: float64(1), object(1)
memory usage: 230.8+ KB


In [9]:
# Convert the 'Date' column to a datetime object
df['Date'] = pd.to_datetime(df['Date'])

# Extract the year from the 'Date' column and create a new column 'Year'
df['Year'] = df['Date'].dt.year

# Group the data by year
grouped = df.groupby('Year')

# Define a function to calculate the upper and lower fences for each group
def iqr_fence(x):
    Q1 = x.quantile(0.25)
    Q3 = x.quantile(0.75)
    IQR = Q3 - Q1
    Lower_Fence = Q1 - (1.5 * IQR)
    Upper_Fence = Q3 + (1.5 * IQR)
    return Lower_Fence, Upper_Fence

# Apply the function to each group to calculate the upper and lower fences
fences = grouped['Price'].apply(iqr_fence)

# Define a function to recode the outliers in each group
def recode_outliers(x):
    year = x.name
    lower_fence, upper_fence = fences.loc[year]
    x[x < lower_fence] = lower_fence
    x[x > upper_fence] = upper_fence
    return x

# Apply the function to each group to recode the outliers
df['Price'] = grouped['Price'].apply(recode_outliers)

# Save the resulting DataFrame to a new CSV file
df.to_csv('E:\Excelr-P269-Internship-Project-Group2-OIl-Price-Prediction-And-Forecasting-Using-Python\Dataset\Cleaned_Oil_Price.csv', index=False)


C:\Users\nazee\AppData\Local\Temp\ipykernel_16148\2444377644.py:31: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df['Price'] = grouped['Price'].apply(recode_outliers)
